# Sistemas inteligentes. Aplicaciones

## Práctica 2-2. Preprocesamiento 2

En esta práctica vas a unir todas las técnicas que hemos visto hasta ahora, para trabajar en un problema de clasificación de textos.

Para realizar esta práctica vamos a utilizar un dataset que contiene documentos de texto ofrecido por scikit_learn. Este dataset se llama [Twenty Newsgroups](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html#sklearn.datasets.fetch_20newsgroups). Su descripción oficial es la siguiente: 

    "The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. To the best of our knowledge, it was originally collected by Ken Lang, probably for his paper “Newsweeder: Learning to filter netnews,” though he does not explicitly mention this collection. The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering."

Para trabajar con este dataset lo primero que debemos hacer es importarlo:

    from sklearn.datasets import fetch_20newsgroups

Una vez importado se leerán los datos de los documentos. Para ello, la llamada al constructor de la clase es la siguiente:, se establece como subset la opción train y de esta forma devuelve los datos de entrenamiento (poner esta opción a test para leer los datos de test).

    twenty_train = fetch_20newsgroups(subset=tipoDatos, shuffle=aleatorio, random_state=semilla, categories=clasesDocumentos)

Los parámetros son los siguientes:
* tipoDatos: string que determina si los datos son de entrenamiento (asignar 'train') o de test (asignar 'test')
* aleatorio: valor booleano que establece si se aleatorizan los datos o no.
* semilla: valor entero que determina la semilla para la generación de números aleatorios. De esta forma los experimentos serán reproducibles.
* clasesDocumentos: lista con los nombre de las clases de los documentos a leer. Si se asigna a None se leen los documentos de todas las clases.

El objeto generado (variable twenty_train) tiene la misma estructura que todos los datasets nativos de Scikit-learn con los que hemos trabajado. Por tanto:
* Los **datos de entrada** correspondientes a los documentos de texto se encuentran en el campo **data**. Por ejemplo, si se desea mostrar el primer documento se puede ejecutar:	print("\n".join(twenty_train.data[0].split("\n")))
* Los **nombres de las clases** se encuentran en el campo **target_names**. En este corpus de documentos se encuentran documentos clasificados en 20 clases diferentes, para visualizarlos ejecuta: print twenty_train.target_names
* Las **clases** de cada documento se encuentran en el campo **target**. Por ejemplo si se quiere obtener la clase del primer documento se ejecutaría: print twenty_train.target[0]
* Si anidamos los campos target_names y target podemos visualizar la clase de cada documento. Por ejemplo si se desea conocer la clase del primer documento se ejecutaría: print twenty_train.target_names[twenty_train.target[0]]

Para simplificar el problema, no vamos a trabajar con todas las clases que existen en el dataset. por contra, solo vamos a quedarnos con las clases: 'alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med'.

Para los datos de dichas clases, debes ser capaz de transformarlos en un dataset tabular y entrenar un modelo de clasificación supervisada que aprenda a determinar la clase de un documento. 

Los pasos a seguir con los vistos en clase hasta el momento:
* preprocesamiento simple del texto
* tokenización
* stemming / lemmatization
* bag of words
* aprender modelo de clasificación (naive-bayes)
* obtener accuracy en train y test

En primer lugar, debes ser capaz de montar un proceso completo de clasificación, desde la lectura de los datos hasta la obtención del accuracy del modelo. Una vez ue tengas un modelo completo funcionando, puedes ir cambiando los diferentes parámetros, para intentar obtener el máximo de accuracy.

In [32]:
from sklearn.datasets import fetch_20newsgroups
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

from nltk import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import re

# --- 1. Carga de datos ---
seed = 12
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=seed, categories=categories)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=seed, categories=categories)

print(f"Documentos de entrenamiento: {len(twenty_train.data)}")
print(f"Documentos de test: {len(twenty_test.data)}")
print(f"Categorías: {twenty_train.target_names}")

# --- 2. Preprocesamiento + Tokenización + Stemming ---
stemmer = SnowballStemmer('english')

def preprocess(text):
    # Eliminar caracteres no alfanuméricos (excepto espacios)
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Tokenizar
    tokens = word_tokenize(text)
    # Stemming
    tokens = [stemmer.stem(t) for t in tokens]
    return ' '.join(tokens)

# Preprocesar todos los documentos de train y test
train_processed = [preprocess(doc) for doc in twenty_train.data]
test_processed = [preprocess(doc) for doc in twenty_test.data]

print(f"\nEjemplo documento original (primeras 200 chars):\n{twenty_train.data[0][:200]}")
print(f"\nEjemplo documento procesado (primeras 200 chars):\n{train_processed[0][:200]}")

# --- 3. Bag of Words ---
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_processed)
X_test = vectorizer.transform(test_processed)

print(f"\nTamaño vocabulario: {len(vectorizer.get_feature_names_out())}")
print(f"Matriz train: {X_train.shape}")
print(f"Matriz test: {X_test.shape}")

# --- 4. Modelo Naive-Bayes ---
y_train = twenty_train.target
y_test = twenty_test.target

clf = MultinomialNB()
clf.fit(X_train, y_train)

# --- 5. Accuracy en train y test ---
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)

print(f"\nAccuracy en TRAIN: {acc_train:.4f}")
print(f"Accuracy en TEST:  {acc_test:.4f}")

print(f"\nClassification Report (Test):\n")
print(classification_report(y_test, y_pred_test, target_names=categories))

[nltk_data] Downloading package punkt to /Users/juancho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/juancho/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/juancho/nltk_data...


Documentos de entrenamiento: 2257
Documentos de test: 1502
Categorías: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

Ejemplo documento original (primeras 200 chars):
From: jr0930@eve.albany.edu (REGAN JAMES P)
Subject: Re: Pascal-Fractals
Organization: State University of New York at Albany
Lines: 10

Apparently, my editor didn't do what I wanted it to do, so I'll

Ejemplo documento procesado (primeras 200 chars):
from jr0930evealbanyedu regan jame p subject re pascalfract organ state univers of new york at albani line 10 appar my editor didnt do what i want it to do so ill tri again im look for ani program or 

Tamaño vocabulario: 30571
Matriz train: (2257, 30571)
Matriz test: (1502, 30571)

Accuracy en TRAIN: 0.9938
Accuracy en TEST:  0.9201

Classification Report (Test):

                        precision    recall  f1-score   support

           alt.atheism       0.88      0.87      0.88       319
soc.religion.christian       0.95      0.94      0.94       